# Cancer data

## 1. total data

In [22]:
# load modules
# basic
import numpy as np
import pandas as pd
from tqdm import tqdm
import itertools

## graph
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import kaleido

## sklearn
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

## statistics
from scipy.stats import pearsonr
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import bartlett
from scipy.stats import levene

In [23]:
# load data
data = pd.read_csv('../data/Cancer/total-cancer-deaths-by-type.csv', encoding='utf-8')
population = pd.read_csv('../data/world_population.csv', encoding='utf-8')

In [24]:
data.isna().sum()

Entity                                                                                     0
Code                                                                                     690
Year                                                                                       0
Deaths - Liver cancer - Sex: Both - Age: All Ages (Number)                                 0
Deaths - Kidney cancer - Sex: Both - Age: All Ages (Number)                                0
Deaths - Lip and oral cavity cancer - Sex: Both - Age: All Ages (Number)                   0
Deaths - Tracheal, bronchus, and lung cancer - Sex: Both - Age: All Ages (Number)          0
Deaths - Larynx cancer - Sex: Both - Age: All Ages (Number)                                0
Deaths - Gallbladder and biliary tract cancer - Sex: Both - Age: All Ages (Number)         0
Deaths - Malignant skin melanoma - Sex: Both - Age: All Ages (Number)                      0
Deaths - Leukemia - Sex: Both - Age: All Ages (Number)                

In [25]:
# checking the data
data.head(10)

,Entity,Code,Year,Deaths - Liver cancer - Sex: Both - Age: All Ages (Number),Deaths - Kidney cancer - Sex: Both - Age: All Ages (Number),Deaths - Lip and oral cavity cancer - Sex: Both - Age: All Ages (Number),"Deaths - Tracheal, bronchus, and lung cancer - Sex: Both - Age: All Ages (Number)",Deaths - Larynx cancer - Sex: Both - Age: All Ages (Number),Deaths - Gallbladder and biliary tract cancer - Sex: Both - Age: All Ages (Number),Deaths - Malignant skin melanoma - Sex: Both - Age: All Ages (Number),...,Deaths - Brain and central nervous system cancer - Sex: Both - Age: All Ages (Number),Deaths - Non-Hodgkin lymphoma - Sex: Both - Age: All Ages (Number),Deaths - Pancreatic cancer - Sex: Both - Age: All Ages (Number),Deaths - Esophageal cancer - Sex: Both - Age: All Ages (Number),Deaths - Testicular cancer - Sex: Both - Age: All Ages (Number),Deaths - Nasopharynx cancer - Sex: Both - Age: All Ages (Number),Deaths - Other pharynx cancer - Sex: Both - Age: All Ages (Number),Deaths - Colon and rectum cancer - Sex: Both - Age: All Ages (Number),Deaths - Non-melanoma skin cancer - Sex: Both - Age: All Ages (Number),Deaths - Mesothelioma - Sex: Both - Age: All Ages (Number)
0,Afghanistan,AFG,1990,851,66,89,983,260,180,47,...,422,996,138,529,3,66,37,539,25,6
1,Afghanistan,AFG,1991,866,66,89,982,263,182,48,...,438,1010,137,535,4,67,37,542,26,7
2,Afghanistan,AFG,1992,890,68,91,989,268,185,51,...,472,1040,137,546,4,69,38,550,26,7
3,Afghanistan,AFG,1993,914,70,93,995,275,189,53,...,505,1062,139,560,5,72,39,555,27,7
4,Afghanistan,AFG,1994,933,71,94,996,282,193,54,...,526,1069,140,575,5,73,39,554,28,8
5,Afghanistan,AFG,1995,946,71,94,995,286,195,56,...,541,1074,139,584,6,74,39,552,28,8
6,Afghanistan,AFG,1996,962,72,95,994,291,198,57,...,559,1081,139,595,6,75,39,550,28,8
7,Afghanistan,AFG,1997,976,73,95,993,295,201,58,...,580,1090,139,606,6,75,39,550,29,8
8,Afghanistan,AFG,1998,985,73,96,990,299,203,59,...,588,1094,140,615,6,76,39,549,29,9
9,Afghanistan,AFG,1999,994,73,96,987,302,206,60,...,591,1098,141,626,6,76,39,548,29,9


In [26]:
# column name 바꾸기
data.columns = data.columns.str.replace("Deaths - ", "").str.replace(" - Sex: Both - Age: All Ages (Number)", "")

In [27]:
data.columns

Index(['Entity', 'Code', 'Year', 'Liver cancer', 'Kidney cancer',
       'Lip and oral cavity cancer', 'Tracheal, bronchus, and lung cancer',
       'Larynx cancer', 'Gallbladder and biliary tract cancer',
       'Malignant skin melanoma', 'Leukemia', 'Hodgkin lymphoma',
       'Multiple myeloma', 'Other neoplasms', 'Breast cancer',
       'Prostate cancer', 'Thyroid cancer', 'Stomach cancer', 'Bladder cancer',
       'Uterine cancer', 'Ovarian cancer', 'Cervical cancer',
       'Brain and central nervous system cancer', 'Non-Hodgkin lymphoma',
       'Pancreatic cancer', 'Esophageal cancer', 'Testicular cancer',
       'Nasopharynx cancer', 'Other pharynx cancer', 'Colon and rectum cancer',
       'Non-melanoma skin cancer', 'Mesothelioma'],
      dtype='object')

In [28]:
# world code = nan, england code = eld, wales code = wls, scotland code = sct,Northland code = nld, Scotland code = sct, Northern Irel
data['Code'][data['Code'] == 'OWID_WRL'] = np.nan
#data['Code'][data['Entity'] == 'Wales'] = 'WLS'
#data['Code'][data['Entity'] == 'England'] = 'ELD'
#data['Code'][data['Entity'] == 'Scotland'] = 'SLD'
#data['Code'][data['Entity'] == 'Northern Ireland'] = 'NIR'

C:\Users\abc\AppData\Local\Temp\ipykernel_12688\2201590237.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### (1) 국가별 확인하기

In [29]:
# 국가 번호 없는 값 제외
country = data[~data['Code'].isna()][['Entity', 'Code', 'Year', 'Liver cancer']]

In [30]:
# 1990 ~ 2019년도 국가별 총 사망자 수
sum_expire = country.groupby(['Entity', 'Code']).sum('Liver cancer').reset_index()

In [31]:
# 상위 10개국
top_10_sum = sum_expire.sort_values(by='Liver cancer', ascending=False).head(10)
top_10_sum

,Entity,Code,Year,Liver cancer
37,China,CHN,60135,6140780
89,Japan,JPN,60135,978747
81,India,IND,60135,606386
180,Thailand,THA,60135,449990
194,United States,USA,60135,413991
168,South Korea,KOR,60135,289401
54,Egypt,EGY,60135,270270
87,Italy,ITA,60135,194128
63,France,FRA,60135,191056
67,Germany,DEU,60135,170726


In [32]:
# 하위 10개국
sum_expire.sort_values(by='Liver cancer').head(10)

,Entity,Code,Year,Liver cancer
130,Niue,NIU,60135,0
183,Tokelau,TKL,60135,0
123,Nauru,NRU,60135,0
189,Tuvalu,TUV,60135,30
155,San Marino,SMR,60135,38
137,Palau,PLW,60135,50
41,Cook Islands,COK,60135,64
156,Sao Tome and Principe,STP,60135,66
151,Saint Kitts and Nevis,KNA,60135,67
3,American Samoa,ASM,60135,71


- 상위 10개국 확인 결과 주로 아시아권 나라들이 차지했으며, 인구수가 가장 많은 중국과 인도가 1, 2위를 차지함
- 인구에 따른 비율로 확인해볼 필요도 있어 보임
- 2019년도 기준 누적 인구수 비율을 확인

### (2) 인구 비율

In [33]:
# 총 인구와 사망 인원수 결합
mergedata = pd.merge(sum_expire, population[population['Year'] == 2019], left_on=['Code'], right_on=['Code'], how='left')
mergedata2 = mergedata.drop(['Entity_y', 'Location code', 'Code-2', 'Index', 'Year_x'], axis=1)
mergedata2.head(10)

,Entity_x,Code,Liver cancer,Year_y,Population
0,Afghanistan,AFG,32122,2019,37212970
1,Albania,ALB,7838,2019,2875637
2,Algeria,DZA,12664,2019,42319750
3,American Samoa,ASM,71,2019,47887
4,Andorra,AND,268,2019,75666
5,Angola,AGO,5745,2019,31808179
6,Antigua and Barbuda,ATG,78,2019,91856
7,Argentina,ARG,22790,2019,44582599
8,Armenia,ARM,5348,2019,2828707
9,Australia,AUS,30253,2019,25170161


In [34]:
# 사망 비율 확인 (사망인구/전체인구 * 10000)
mergedata2['death_per_population'] = round(mergedata2['Liver cancer']/(mergedata2['Population'])*10000, 2)

In [35]:
# 상위 10개국
top_10_perc = mergedata2.sort_values(by='death_per_population', ascending=False).head(10)
top_10_perc

,Entity_x,Code,Liver cancer,Year_y,Population,death_per_population
117,Mongolia,MNG,41187,2019,3198396,128.77
89,Japan,JPN,978747,2019,126040583,77.65
180,Thailand,THA,449990,2019,71226152,63.18
168,South Korea,KOR,289401,2019,51749531,55.92
184,Tonga,TON,525,2019,105028,49.99
116,Monaco,MCO,161,2019,37035,43.47
37,China,CHN,6140780,2019,1419730043,43.25
41,Cook Islands,COK,64,2019,17179,37.25
74,Guinea,GIN,47159,2019,12713491,37.09
177,Taiwan,TWN,86625,2019,23754579,36.47


### (3) 그래프 생성하기

In [36]:
# top 10 graphs
fig = go.Figure()

colors = ['lightslategray','lightslategray','lightslategray',
          'lightslategray','lightslategray','red',
          'lightslategray','lightslategray','lightslategray',
          'lightslategray']

fig.add_trace(go.Bar(x = top_10_sum['Entity'],
                     y = top_10_sum['Liver cancer'],
                     marker_color=colors,
                     opacity=1))

fig.update_layout(title_text="1990년 ~ 2019년 간암 사망자 누적 인원수",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff',
                  xaxis=dict(title='Country'),
                  yaxis=dict(title='Count(명)'))

fig.update_traces(#marker_color= 히스토그램 색, 
                 #marker_line_width=히스토그램 테두리 두깨,                            
                 #marker_line_color=히스토그램 테두리 색,
                 marker_opacity = 0.4,
                 )
fig.show()

# 저장
fig.write_html("../figure/cancer_death_count.html")

In [37]:
# top 10 percentage graphs
fig = go.Figure()

colors = ['lightslategray','lightslategray','lightslategray',
          'red','lightslategray','lightslategray',
          'lightslategray','lightslategray','lightslategray',
          'lightslategray']

fig.add_trace(go.Bar(x = top_10_perc['Entity_x'],
                     y = top_10_perc['death_per_population'],
                     marker_color=colors,
                     opacity=0.8))

fig.update_layout(title_text="2019년 간암 사망자 비율",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff',
                  xaxis=dict(title='Country'),
                  yaxis=dict(title='Count(10000명당 명수)'))

fig.update_traces(#marker_color= 히스토그램 색, 
                 #marker_line_width=히스토그램 테두리 두깨,                            
                 #marker_line_color=히스토그램 테두리 색,
                 marker_opacity = 0.4,
                 )

fig.show()

# 저장
fig.write_image("../figure/cancer_death_count_perc.png")

- 두 그래프를 확인했을 때 한국인의 간암 사망 인원 비율이 높은 것을 확인할 수 있었음
- 간에 관한 질병에 대해 탐색할 수 있는 발판이 될 수 있는 자료

### (4) 애니메이션 그래프 생성하기

In [38]:
# top10 국가 이름 뽑기
top10 = list(top_10_sum['Entity'])

In [39]:
# top10 데이터 가져오기
top10_liver = country[country['Entity'].apply(lambda x: x in top10)]
top10_liver

,Entity,Code,Year,Liver cancer
1140,China,CHN,1990,232449
1141,China,CHN,1991,244908
1142,China,CHN,1992,256415
1143,China,CHN,1993,268173
1144,China,CHN,1994,279913
...,...,...,...,...
6355,United States,USA,2015,20428
6356,United States,USA,2016,21382
6357,United States,USA,2017,21981
6358,United States,USA,2018,22833


In [40]:
# pivot table
top10_years = pd.pivot_table(top10_liver, values='Liver cancer',
                index = ['Entity', 'Code'],
                columns = ['Year'],
                    aggfunc=np.sum,
                    fill_value = 0).reset_index()
top10_years.columns = list(top10_years.columns[:2]) + ['Y' + str(value) for value in top10_years.columns[2:]]
top10_years = top10_years.sort_values(by='Y2019', ascending=False)
top10_years

,Entity,Code,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,...,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019
0,China,CHN,232449,244908,256415,268173,279913,289004,295270,296720,...,141707,140688,142470,148066,156909,164943,172587,178251,182139,187700
6,Japan,JPN,19987,21213,22768,24566,26455,29563,31603,33379,...,34720,34350,33877,33343,32818,32511,32467,32622,33708,34515
4,India,IND,11859,12225,12563,13085,13524,13875,14560,15496,...,23700,24806,25115,25939,26136,27431,28189,28930,29844,30709
8,Thailand,THA,7883,8361,8766,9070,9431,9781,10135,9941,...,18545,19431,20036,20334,20798,21338,22204,23015,23759,24527
9,United States,USA,6454,6729,7085,7499,7897,8329,8712,9120,...,16713,17462,18108,18761,19541,20428,21382,21981,22833,23807
7,South Korea,KOR,3413,3289,3171,3092,3099,3088,3699,4999,...,12264,12308,12421,12477,12580,12734,12956,13489,13910,14477
1,Egypt,EGY,5198,5292,5202,5325,5464,5529,5651,5792,...,11979,12295,12521,12541,12784,13133,13304,13337,13451,13590
2,France,FRA,4009,4253,4502,4781,5120,5342,5594,5773,...,7206,7343,7474,7545,7563,7713,7644,7732,7718,7792
3,Germany,DEU,3082,3146,3217,3334,3472,3571,3733,3908,...,7049,7040,7077,7125,7204,7395,7522,7565,7643,7743
5,Italy,ITA,5829,6095,6317,6603,6777,6711,6767,6750,...,6398,6630,6829,6831,6824,6787,6402,6647,6790,6919


In [41]:
# Bar Plot 생성
fig = go.Figure()

colors = ['lightslategray','lightslategray','lightslategray',
          'lightslategray','lightslategray','red',
          'lightslategray','lightslategray','lightslategray',
          'lightslategray']

# 초기 프레임 추가
fig.add_trace(go.Bar(x=top10_years['Entity'], y=top10_years['Y1990'],
                     marker_color=colors,
                     name='Y1990'))

# 추가 프레임 및 애니메이션 설정
for column in top10_years.columns[3:]:
    fig.add_trace(go.Bar(x=top10_years['Entity'], y=top10_years[column], visible=False))

fig.update_layout(updatemenus=[dict(type='buttons', showactive=False, buttons=[
    dict(label='Play',
         method='animate',
         args=[None, dict(frame=dict(duration=500, redraw=True), fromcurrent=True)]),
    dict(label='Pause',
         method='animate',
         args=[[None], dict(frame=dict(duration=0, redraw=False), mode='immediate')],
         ),
    dict(label='Reset',
         method='animate',
         args=[None, dict(frame=dict(duration=0, redraw=True), fromcurrent=True)],
         ),
])])

# 애니메이션 프레임 설정
frames = [go.Frame(data=[go.Bar(x=top10_years['Entity'], y=top10_years[column], name=column)])
          for column in top10_years.columns[3:]]
fig.frames = frames

# Y축 범위 고정
fig.update_yaxes(range=[0, 50000])  # 원하는 범위로 수정

# 그래프 레이아웃 설정
fig.update_layout(title_text="1990년 ~ 2019년 Top 10 간암 사망자 수 변화",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff',
                  xaxis=dict(title='Country'),
                  yaxis=dict(title='Count(명)'),
                  showlegend=True)

fig.update_traces(#marker_color= 히스토그램 색, 
                 #marker_line_width=히스토그램 테두리 두깨,                            
                 #marker_line_color=히스토그램 테두리 색,
                 marker_opacity = 0.4,
                 )
# 그래프 출력
fig.show()

# 저장
fig.write_image("../figure/cancer_death_count_ani.png")

- 미비하지만 경향을 보이는 것 같음.

In [42]:
# 누적 데이터 만들기
country['cumsumliver'] = country.groupby(['Entity', 'Code'])['Liver cancer'].cumsum()
country

,Entity,Code,Year,Liver cancer,cumsumliver
0,Afghanistan,AFG,1990,851,851
1,Afghanistan,AFG,1991,866,1717
2,Afghanistan,AFG,1992,890,2607
3,Afghanistan,AFG,1993,914,3521
4,Afghanistan,AFG,1994,933,4454
...,...,...,...,...,...
6835,Zimbabwe,ZWE,2015,948,23425
6836,Zimbabwe,ZWE,2016,966,24391
6837,Zimbabwe,ZWE,2017,982,25373
6838,Zimbabwe,ZWE,2018,998,26371


In [43]:
# data merge
mergedata3 = pd.merge(country, population.iloc[:, 2:], left_on=['Code', 'Year'], right_on=['Code', 'Year'], how='left')
mergedata4 = mergedata3.drop(['Location code', 'Code-2', 'Liver cancer'], axis=1)
mergedata4.head(10)

,Entity,Code,Year,cumsumliver,Population
0,Afghanistan,AFG,1990,851,10845924
1,Afghanistan,AFG,1991,1717,10543668
2,Afghanistan,AFG,1992,2607,10946666
3,Afghanistan,AFG,1993,3521,13168200
4,Afghanistan,AFG,1994,4454,14839320
5,Afghanistan,AFG,1995,5400,16071789
6,Afghanistan,AFG,1996,6362,16766035
7,Afghanistan,AFG,1997,7338,17447156
8,Afghanistan,AFG,1998,8323,18130482
9,Afghanistan,AFG,1999,9317,18855782


In [44]:
# top10 국가 이름 뽑기
top10_country2 = top_10_perc['Entity_x']
top10_country2.name = 'Entity'
top10_country2


117        Mongolia
89            Japan
180        Thailand
168     South Korea
184           Tonga
116          Monaco
37            China
41     Cook Islands
74           Guinea
177          Taiwan
Name: Entity, dtype: object

In [45]:
# top10 데이터 가져오기
top10_liverp = mergedata4[mergedata4['Entity'].apply(lambda x: x in list(top10_country2))]
top10_liverp['death_per_population'] = round(top10_liverp['cumsumliver']/(top10_liverp['Population'])*10000, 2)
top10_liverp
tp = top10_liverp[['Entity', 'Year', 'death_per_population']]

C:\Users\abc\AppData\Local\Temp\ipykernel_12688\3435563571.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [46]:
# pivot table
top10_yearsp = pd.pivot_table(tp, values='death_per_population',
                index = ['Entity'],
                columns = ['Year'],
                    aggfunc=np.sum,
                    fill_value = 0).reset_index()
top10_yearsp.columns = list(top10_yearsp.columns[:1]) + ['Y' + str(value) for value in top10_yearsp.columns[1:]]
top10_yearsp = top10_yearsp.sort_values(by='Y2019', ascending=False)
top10_yearsp

,Entity,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,...,Y2010,Y2011,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019
5,Mongolia,3.24,6.45,9.58,12.67,15.73,18.81,21.92,25.03,28.20,...,83.69,89.39,94.73,99.93,104.92,109.75,114.56,119.31,124.05,128.77
3,Japan,1.62,3.33,5.15,7.10,9.20,11.54,14.03,16.64,19.40,...,52.96,55.66,58.37,61.05,63.72,66.38,69.05,71.78,74.66,77.65
8,Thailand,1.44,2.92,4.43,5.94,7.48,9.03,10.60,12.11,13.66,...,37.40,40.00,42.65,45.31,48.03,50.82,53.76,56.76,59.90,63.18
6,South Korea,0.78,1.51,2.21,2.89,3.57,4.23,5.02,6.08,7.53,...,35.34,37.67,39.83,41.96,44.08,46.19,48.34,50.76,53.28,55.92
9,Tonga,1.42,2.83,4.24,5.63,7.13,8.61,10.09,11.67,13.17,...,32.73,34.41,36.12,38.02,39.96,41.92,43.93,45.95,47.97,49.99
4,Monaco,0.66,1.31,1.96,2.59,3.21,4.14,5.06,5.97,7.17,...,29.89,31.28,32.63,33.93,35.22,36.49,37.76,39.66,41.60,43.47
0,China,2.03,4.10,6.23,8.42,10.67,12.95,15.25,17.54,19.79,...,34.73,35.54,36.36,37.17,38.06,39.00,40.02,41.04,42.09,43.25
1,Cook Islands,1.17,2.32,3.45,4.57,5.68,6.78,7.88,8.98,10.38,...,24.82,25.14,25.50,26.72,27.98,29.27,31.20,33.21,35.21,37.25
2,Guinea,1.73,3.37,4.97,6.57,8.08,9.62,11.09,12.57,14.18,...,29.94,30.90,31.83,32.74,33.60,34.41,35.14,35.82,36.48,37.09
7,Taiwan,0.99,2.03,3.13,4.26,5.42,6.60,7.81,8.94,10.23,...,27.69,28.75,29.68,30.56,31.45,32.34,33.29,34.29,35.35,36.47


In [47]:
# Bar Plot 생성
fig = go.Figure()

colors = ['lightslategray','lightslategray','lightslategray',
          'red', 'lightslategray','lightslategray',
          'lightslategray','lightslategray','lightslategray',
          'lightslategray']

# 초기 프레임 추가
fig.add_trace(go.Bar(x=top10_yearsp['Entity'], y=top10_yearsp['Y1990'],
                     marker_color=colors,
                     name='Y1990'))

# 추가 프레임 및 애니메이션 설정
for column in top10_yearsp.columns[2:]:
    fig.add_trace(go.Bar(x=top10_yearsp['Entity'], y=top10_yearsp[column], visible=False))

fig.update_layout(updatemenus=[dict(type='buttons', showactive=False, buttons=[
    dict(label='Play',
         method='animate',
         args=[None, dict(frame=dict(duration=500, redraw=True), fromcurrent=True)]),
    dict(label='Pause',
         method='animate',
         args=[[None], dict(frame=dict(duration=0, redraw=False), mode='immediate')],
         ),
    dict(label='Reset',
         method='animate',
         args=[None, dict(frame=dict(duration=0, redraw=True), fromcurrent=True)],
         ),
])])

# 애니메이션 프레임 설정
frames = [go.Frame(data=[go.Bar(x=top10_yearsp['Entity'], y=top10_yearsp[column], name=column)])
          for column in top10_yearsp.columns[3:]]
fig.frames = frames

# Y축 범위 고정
fig.update_yaxes(range=[0, 100])  # 원하는 범위로 수정

# 그래프 레이아웃 설정
fig.update_layout(title_text="1990년 ~ 2019년 Top 10 간암 사망자 비율 변화",
                  title_x = 0.5,
                  title_xanchor = 'center',
                  title_font_size = 25,
                  title_font_color = 'black',
                  title_font_family = 'NanumSquare',
                  plot_bgcolor='#ffffff',
                  xaxis=dict(title='Country'),
                  yaxis=dict(title='Count(10000명당 명수)'),
                  showlegend=True)

fig.update_traces(#marker_color= 히스토그램 색, 
                 #marker_line_width=히스토그램 테두리 두깨,                            
                 #marker_line_color=히스토그램 테두리 색,
                 marker_opacity = 0.4,
                 )
# 그래프 출력
fig.show()

# 저장
fig.write_image("../figure/cancer_death_count_perc_ani.png")